<a href="https://colab.research.google.com/github/Fharuk/DATATALKS-2025-COHORT/blob/main/Datatalks_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
response = requests.get(url)
with open("car_fuel_efficiency.csv", "wb") as f:
    f.write(response.content)

In [2]:
import pandas as pd

df = pd.read_csv('car_fuel_efficiency.csv')

display(df.head())
display(df.info())

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   num_cylinders        9222 non-null   float64
 2   horsepower           8996 non-null   float64
 3   vehicle_weight       9704 non-null   float64
 4   acceleration         8774 non-null   float64
 5   model_year           9704 non-null   int64  
 6   origin               9704 non-null   object 
 7   fuel_type            9704 non-null   object 
 8   drivetrain           9704 non-null   object 
 9   num_doors            9202 non-null   float64
 10  fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 834.1+ KB


None

In [3]:
df = df.fillna(0)
display(df.isnull().sum())

,0
engine_displacement,0
num_cylinders,0
horsepower,0
vehicle_weight,0
acceleration,0
model_year,0
origin,0
fuel_type,0
drivetrain,0
num_doors,0


In [4]:
from sklearn.model_selection import train_test_split

df_train, df_temp = train_test_split(df, test_size=0.4, random_state=1)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=1)

print("Shape of training set:", df_train.shape)
print("Shape of validation set:", df_val.shape)
print("Shape of test set:", df_test.shape)

Shape of training set: (5822, 11)
Shape of validation set: (1941, 11)
Shape of test set: (1941, 11)


In [5]:
from sklearn.feature_extraction import DictVectorizer

train_dicts = df_train.to_dict(orient='records')
val_dicts = df_val.to_dict(orient='records')
test_dicts = df_test.to_dict(orient='records')

In [6]:
dv = DictVectorizer(sparse=True)

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(5822, 15)
(1941, 15)
(1941, 15)


In [7]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Separate target variable
y_train = df_train['fuel_efficiency_mpg']
y_val = df_val['fuel_efficiency_mpg']
y_test = df_test['fuel_efficiency_mpg']

# Drop the target variable from features
X_train_features = dv.transform(df_train.drop('fuel_efficiency_mpg', axis=1).to_dict(orient='records'))
X_val_features = dv.transform(df_val.drop('fuel_efficiency_mpg', axis=1).to_dict(orient='records'))
X_test_features = dv.transform(df_test.drop('fuel_efficiency_mpg', axis=1).to_dict(orient='records'))


# Train the Decision Tree Regressor
dt_model = DecisionTreeRegressor(max_depth=1, random_state=1)
dt_model.fit(X_train_features, y_train)

# Get the feature used for splitting
feature_index = dt_model.tree_.feature[0]
feature_name = dv.get_feature_names_out()[feature_index]

print(f"The feature used for splitting the data is: {feature_name}")

The feature used for splitting the data is: vehicle_weight


In [8]:
from sklearn.ensemble import RandomForestRegressor

# Train the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf_model.fit(X_train_features, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [9]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred_val = rf_model.predict(X_val_features)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"The RMSE of the Random Forest model on the validation data is: {rmse}")

The RMSE of the Random Forest model on the validation data is: 0.4603949847477043


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

rmse_scores = {}

for n_estimators in range(10, 201, 10):
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=1, n_jobs=-1)
    rf_model.fit(X_train_features, y_train)
    y_pred_val = rf_model.predict(X_val_features)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    rmse_scores[n_estimators] = rmse
    print(f"n_estimators: {n_estimators}, RMSE: {rmse}")

n_estimators: 10, RMSE: 0.4603949847477043
n_estimators: 20, RMSE: 0.445726517255611
n_estimators: 30, RMSE: 0.44003455503795696
n_estimators: 40, RMSE: 0.4389267433835154
n_estimators: 50, RMSE: 0.43787442613142724
n_estimators: 60, RMSE: 0.43624531255828136
n_estimators: 70, RMSE: 0.4365982420023339
n_estimators: 80, RMSE: 0.4366565359880605
n_estimators: 90, RMSE: 0.43614114579497304
n_estimators: 100, RMSE: 0.43605444202253013
n_estimators: 110, RMSE: 0.435681242395639
n_estimators: 120, RMSE: 0.4361483712268003
n_estimators: 130, RMSE: 0.43592318303167915
n_estimators: 140, RMSE: 0.4360466214191877
n_estimators: 150, RMSE: 0.4361645652902668
n_estimators: 160, RMSE: 0.43612600943180324
n_estimators: 170, RMSE: 0.43616962221565925
n_estimators: 180, RMSE: 0.43588704022212754
n_estimators: 190, RMSE: 0.4360170834045449
n_estimators: 200, RMSE: 0.435580534788385


In [13]:
# Find the n_estimators value where RMSE improvement stops
prev_rmse = float('inf')
stop_value = 200

for n_estimators, rmse in rmse_scores.items():
    if prev_rmse - rmse < 0.001:
        stop_value = n_estimators
        break
    prev_rmse = rmse

print(f"RMSE stops improving after n_estimators: {stop_value}")

RMSE stops improving after n_estimators: 70


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

max_depths = [10, 15, 20, 25]
n_estimators_values = range(10, 201, 10)
results = {}

for max_depth in max_depths:
    rmse_scores = []
    for n_estimators in n_estimators_values:
        rf_model = RandomForestRegressor(n_estimators=n_estimators,
                                         max_depth=max_depth,
                                         random_state=1,
                                         n_jobs=-1)
        rf_model.fit(X_train_features, y_train)
        y_pred_val = rf_model.predict(X_val_features)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
        rmse_scores.append(rmse)
    mean_rmse = np.mean(rmse_scores)
    results[max_depth] = mean_rmse
    print(f"Max Depth: {max_depth}, Mean RMSE: {mean_rmse}")

# Find the best max_depth
best_max_depth = min(results, key=results.get)
print(f"The best max_depth is: {best_max_depth}")

Max Depth: 10, Mean RMSE: 0.4365198051862447


KeyboardInterrupt: 

In [15]:
# Train the Random Forest Regressor with specified parameters
rf_model_best = RandomForestRegressor(n_estimators=10,
                                     max_depth=20,
                                     random_state=1,
                                     n_jobs=-1)
rf_model_best.fit(X_train_features, y_train)

RandomForestRegressor(max_depth=20, n_estimators=10, n_jobs=-1, random_state=1)

In [17]:
# Get feature importances
feature_importances = rf_model_best.feature_importances_

# Get feature names from the DictVectorizer
feature_names = dv.get_feature_names_out()

# Create a pandas Series for better visualization
importances_series = pd.Series(feature_importances, index=feature_names)

# Sort feature importances in descending order
sorted_importances = importances_series.sort_values(ascending=False)

# Display the sorted feature importances
print("Feature Importances:")
display(sorted_importances)

Feature Importances:


,0
vehicle_weight,0.959843
horsepower,0.015952
acceleration,0.011432
engine_displacement,0.003149
model_year,0.003099
num_cylinders,0.002318
num_doors,0.001582
origin=USA,0.000481
origin=Asia,0.000467
origin=Europe,0.000413


In [18]:
%pip install xgboost

In [19]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train_features, label=y_train)
dval = xgb.DMatrix(X_val_features, label=y_val)

In [20]:
xgb_params_03 = {
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'eta': 0.3
}

watchlist = [(dtrain, 'train'), (dval, 'val')]

model_03 = xgb.train(xgb_params_03,
                     dtrain,
                     num_boost_round=100,
                     evals=watchlist,
                     verbose_eval=10)

[0]	train-rmse:1.83282	val-rmse:1.82567
[10]	train-rmse:0.37400	val-rmse:0.43004
[20]	train-rmse:0.33724	val-rmse:0.42509
[30]	train-rmse:0.31757	val-rmse:0.42754
[40]	train-rmse:0.29886	val-rmse:0.43086
[50]	train-rmse:0.28277	val-rmse:0.43405
[60]	train-rmse:0.26525	val-rmse:0.43583
[70]	train-rmse:0.24983	val-rmse:0.43777
[80]	train-rmse:0.23379	val-rmse:0.43902
[90]	train-rmse:0.22023	val-rmse:0.44154
[99]	train-rmse:0.20896	val-rmse:0.44340


In [21]:
xgb_params_01 = {
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'eta': 0.1
}

watchlist = [(dtrain, 'train'), (dval, 'val')]

model_01 = xgb.train(xgb_params_01,
                     dtrain,
                     num_boost_round=100,
                     evals=watchlist,
                     verbose_eval=10)

[0]	train-rmse:2.31334	val-rmse:2.30592
[10]	train-rmse:0.91846	val-rmse:0.92183
[20]	train-rmse:0.49316	val-rmse:0.52039
[30]	train-rmse:0.38552	val-rmse:0.43528
[40]	train-rmse:0.35546	val-rmse:0.41889
[50]	train-rmse:0.34115	val-rmse:0.41644
[60]	train-rmse:0.33254	val-rmse:0.41610
[70]	train-rmse:0.32553	val-rmse:0.41617
[80]	train-rmse:0.32012	val-rmse:0.41654
[90]	train-rmse:0.31718	val-rmse:0.41654
[99]	train-rmse:0.31183	val-rmse:0.41674


In [23]:
import xgboost as xgb

# Retrain model with eta=0.3 and capture evaluation results
xgb_params_03 = {
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'eta': 0.3
}

watchlist = [(dtrain, 'train'), (dval, 'val')]

model_03_evals = {}
model_03 = xgb.train(xgb_params_03,
                     dtrain,
                     num_boost_round=100,
                     evals=watchlist,
                     evals_result=model_03_evals,
                     verbose_eval=False)

# Retrain model with eta=0.1 and capture evaluation results
xgb_params_01 = {
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'eta': 0.1
}

model_01_evals = {}
model_01 = xgb.train(xgb_params_01,
                     dtrain,
                     num_boost_round=100,
                     evals=watchlist,
                     evals_result=model_01_evals,
                     verbose_eval=False)


# Extract validation RMSE for the last round (round 99 as boosting rounds are 0-indexed)
final_rmse_03 = model_03_evals['val']['rmse'][-1]
final_rmse_01 = model_01_evals['val']['rmse'][-1]

# Print the final validation RMSE for both models
print(f"Final Validation RMSE for eta=0.3: {final_rmse_03}")
print(f"Final Validation RMSE for eta=0.1: {final_rmse_01}")

# Compare the results
if final_rmse_01 < final_rmse_03:
    print("The model with eta=0.1 performed better on the validation set.")
elif final_rmse_03 < final_rmse_01:
    print("The model with eta=0.3 performed better on the validation set.")
else:
    print("Both models performed equally well on the validation set.")

Final Validation RMSE for eta=0.3: 0.4434046335707294
Final Validation RMSE for eta=0.1: 0.4167428747828132
The model with eta=0.1 performed better on the validation set.
